In [2]:
# tag means classes
# patterns means what user is going to ask
# response is chatbot response

data = {"intents":[
    {"tag":"greeting",
    "patterns":["Hello", "How are you?", "Hi There", "Hi", "What's up"],
    "responses":["Howdy Partner!", "Hello", "How are you doing?", "Greetings!", "How do you do"]
    },
    {"tag":"age",
    "patterns":["how old are you", "when is your birthday", "when was you born"],
    "responses":["I am 24 years old", "I was born in 1996", "My birthday is May 23 and I was born in 2003", "23/05/2003"]
    },
    {"tag":"date",
    "patterns":["what are you doing this weeekend?", "do you want to hangout sometime?", "what are your plans for this week"],
    "responses":["I am available all week", "I don't have any plans", "I am not busy"]
    },
    {"tag":"name",
    "patterns":["what's your name", "what are you called", "who are you"],
    "responses":["My name is kippi", "I am Kippi", "Kippi"]
    },
    {"tag":"goodbye",
    "patterns":["bye", "g2g", "see ya", "adios", "cya"],
    "responses":["It was nice speaking to you", "See you later", "Speak soon"]
    },
    
]}

In [3]:
# For each tag we created, we would specify patterns, Essentially this defines the different ways of
# how a user may pose a query to the chatbot.

# The chatbot would then take these patterns and use them as training data to determine what someone
# asking for chatbot name would like so that it could adopt to different ways someone may ask to know the bot's name.

In [8]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to D:\venv\MLvenv\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to D:\venv\MLvenv\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [5]:
# In order to create our training data below steps to be followed:

# create a vocabulary of all the words used in the patterns
# create a list of the classes - tags of each intent
# create a list of all the patterns within the intents file
# create a list of all the associates tags to go with each patterns in the intents file

In [9]:
# initialising Lemmatizer to get stems of words

lemmatizer = WordNetLemmatizer()

words = []
classes = []
doc_x = []
doc_y = []

In [6]:
# Loop through all the intents
# tokenize each pattern and append token to words, the patterns and the
# associated tag to their associated list

In [11]:
for intent in data["intents"]:
    for pattern in intent['patterns']:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_x.append(pattern)
        doc_y.append(intent['tag'])
    # add tag to the classes
    if intent["tag"] not in classes:
        classes.append(intent['tag'])

In [12]:
# Lemmatize all the words in the vocab and convert them to lowercase

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

In [14]:
# sorting the vocab and classes in alphabetical order and taking the set to ensure no duplicates occur

words = sorted(set(words))
classes = sorted(set(classes))

In [15]:
print(words)

["'s", 'adios', 'are', 'birthday', 'born', 'bye', 'called', 'cya', 'do', 'doing', 'for', 'g2g', 'hangout', 'hello', 'hi', 'how', 'is', 'name', 'old', 'plan', 'see', 'sometime', 'there', 'this', 'to', 'up', 'wa', 'want', 'weeekend', 'week', 'what', 'when', 'who', 'ya', 'you', 'your']


In [16]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [17]:
print(doc_x)

['Hello', 'How are you?', 'Hi There', 'Hi', "What's up", 'how old are you', 'when is your birthday', 'when was you born', 'what are you doing this weeekend?', 'do you want to hangout sometime?', 'what are your plans for this week', "what's your name", 'what are you called', 'who are you', 'bye', 'g2g', 'see ya', 'adios', 'cya']


In [18]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


In [19]:
# List for training data

training = []
out_empty = [0]*len(classes)

# creating bag of words model

for idx, doc in enumerate(doc_x):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    
    training.append([bow, output_row])

random.shuffle(training)

training = np.array(training, dtype = object)

train_x = np.array(list(training[:,0]))
train_y = np.array(list(training[:,1]))

In [7]:
# The model will look at the features and predict the tag associated with the features
# then will select an appropriate message/response from the tag

In [20]:
input_shape = (len(train_x[0]),)
output_shape = len(train_y[0])
epochs = 200

In [25]:
model = Sequential()
model.add(Dense(128, input_shape = input_shape, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = 'softmax'))
adam = tf.keras.optimizers.Adam(learning_rate = 0.01, decay = 1e-6)

model.compile(loss = 'categorical_crossentropy',
             optimizer = adam,
             metrics = ['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               4736      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 13,317
Trainable params: 13,317
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
model.fit(x = train_x, y = train_y, epochs = 200, verbose = 1)

Epoch 1/200
1/1 [==============================] - 1s 529ms/step - loss: 1.6687 - accuracy: 0.1579
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 1.4973 - accuracy: 0.4737
Epoch 3/200
1/1 [==============================] - 0s 9ms/step - loss: 1.3908 - accuracy: 0.4737
Epoch 4/200
1/1 [==============================] - 0s 9ms/step - loss: 1.3970 - accuracy: 0.5789
Epoch 5/200
1/1 [==============================] - 0s 9ms/step - loss: 1.2619 - accuracy: 0.6842
Epoch 6/200
1/1 [==============================] - 0s 7ms/step - loss: 1.1286 - accuracy: 0.6316
Epoch 7/200
1/1 [==============================] - 0s 7ms/step - loss: 1.0091 - accuracy: 0.6842
Epoch 8/200
1/1 [==============================] - 0s 5ms/step - loss: 0.9079 - accuracy: 0.8947
Epoch 9/200
1/1 [==============================] - 0s 9ms/step - loss: 0.7465 - accuracy: 0.7895
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 0.6123 - accuracy: 0.8947
Epoch 11/200
1/1 [=========

1/1 [==============================] - 0s 5ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 85/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 86/200
1/1 [==============================] - 0s 7ms/step - loss: 9.7926e-04 - accuracy: 1.0000
Epoch 87/200
1/1 [==============================] - 0s 4ms/step - loss: 7.8672e-04 - accuracy: 1.0000
Epoch 88/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 89/200
1/1 [==============================] - 0s 10ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 90/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 91/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 92/200
1/1 [==============================] - 0s 10ms/step - loss: 1.8882e-04 - accuracy: 1.0000
Epoch 93/200
1/1 [==============================] - 0s 8ms/step - loss: 4.7082e-04 - accuracy: 1.0000
Epoch 94/200
1/

1/1 [==============================] - 0s 4ms/step - loss: 6.2219e-04 - accuracy: 1.0000
Epoch 166/200
1/1 [==============================] - 0s 5ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 4ms/step - loss: 6.9159e-05 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 5ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 6ms/step - loss: 8.1778e-05 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 5ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 2ms/step - loss: 8.8884e-04 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 6ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 17

In [31]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0]*len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            return np.array(bow)

In [34]:
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
    
    y_pred.sort(key = lambda x:x[1], reverse = True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_responses(intents_list, intents_json):
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [ ]:
# running the chatbot

while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_responses(intents, data)
    print(result)

hello
1/1 [==============================] - 0s 16ms/step
Speak soon
what is your name
1/1 [==============================] - 0s 16ms/step
Speak soon
